In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

In [2]:
modeldata = pd.read_csv('Finaldf_Model.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
modeldata = modeldata.sort_values(by=['EquipmentID', 'EventTimeStamp'])

In [5]:
modeldata.head()

,EventTimeStamp,EquipmentID,Target,Engine_Feature,Descriptor_Fuel,Engine_Exhaust,Engine_Coolant,Wheel_Sensor,NOx_limits
65866,2015-07-21 13:53:33,301,0,0,0,0,0,0,0
75194,2015-08-02 13:50:14,301,0,0,0,0,0,0,0
80483,2015-08-09 06:31:51,301,0,1,0,0,0,0,0
80546,2015-08-09 09:27:04,301,0,1,0,0,0,0,0
85043,2015-08-13 15:48:03,301,0,0,0,0,0,0,0


In [0]:
modeldata['EventTimeStamp'] = pd.to_datetime(modeldata['EventTimeStamp'])

In [0]:
modeldata['Date'] = modeldata.EventTimeStamp.dt.date

In [10]:
modeldata.head()

,EventTimeStamp,EquipmentID,Target,Engine_Feature,Descriptor_Fuel,Engine_Exhaust,Engine_Coolant,Wheel_Sensor,NOx_limits,Date
65866,2015-07-21 13:53:33,301,0,0,0,0,0,0,0,2015-07-21
75194,2015-08-02 13:50:14,301,0,0,0,0,0,0,0,2015-08-02
80483,2015-08-09 06:31:51,301,0,1,0,0,0,0,0,2015-08-09
80546,2015-08-09 09:27:04,301,0,1,0,0,0,0,0,2015-08-09
85043,2015-08-13 15:48:03,301,0,0,0,0,0,0,0,2015-08-13


In [0]:
X=modeldata[['Engine_Feature', 'Descriptor_Fuel', 'Engine_Exhaust', 'Engine_Coolant', 'Wheel_Sensor', 'NOx_limits']]
y=modeldata['Target']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify= y)

In [0]:
clf=RandomForestClassifier(n_estimators=50)

In [0]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [0]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
#random train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify)


In [12]:
failed = modeldata[modeldata['Target']==1]
failed.head()

,EventTimeStamp,EquipmentID,Target,Engine_Feature,Descriptor_Fuel,Engine_Exhaust,Engine_Coolant,Wheel_Sensor,NOx_limits,Date
541073,2020-01-06 07:20:54,302,1,1,0,0,0,0,0,2020-01-06
541074,2020-01-06 07:20:54,302,1,1,0,0,0,0,0,2020-01-06
186101,2016-02-15 06:38:41,304,1,1,0,0,0,0,0,2016-02-15
186102,2016-02-15 06:39:20,304,1,0,0,0,0,0,0,2016-02-15
186180,2016-02-15 08:15:31,304,1,0,0,0,0,0,0,2016-02-15


In [16]:
failed['ID_Date'] = failed.EquipmentID.astype('str') + '_' + failed.Date.astype('str')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
unique_failures = failed.ID_Date.unique().tolist()

In [0]:
X_train, X_test = train_test_split(unique_failures, test_size=0.2)

In [27]:
len(X_train)

1737

In [28]:
len(unique_failures)

2172

In [0]:
non_failed = modeldata[modeldata['Target']==0]


In [30]:
non_failed['ID_Date'] = non_failed.EquipmentID.astype('str') + '_' + non_failed.Date.astype('str')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
non_failed[~non_failed['ID_Date'].isin(unique_failures)]

,EventTimeStamp,EquipmentID,Target,Engine_Feature,Descriptor_Fuel,Engine_Exhaust,Engine_Coolant,Wheel_Sensor,NOx_limits,Date,ID_Date
65866,2015-07-21 13:53:33,301,0,0,0,0,0,0,0,2015-07-21,301_2015-07-21
75194,2015-08-02 13:50:14,301,0,0,0,0,0,0,0,2015-08-02,301_2015-08-02
80483,2015-08-09 06:31:51,301,0,1,0,0,0,0,0,2015-08-09,301_2015-08-09
80546,2015-08-09 09:27:04,301,0,1,0,0,0,0,0,2015-08-09,301_2015-08-09
85043,2015-08-13 15:48:03,301,0,0,0,0,0,0,0,2015-08-13,301_2015-08-13
...,...,...,...,...,...,...,...,...,...,...,...
484486,2018-11-21 20:57:59,310,0,0,0,0,0,0,0,2018-11-21,310_2018-11-21
1980,2015-02-24 13:45:06,R1762,0,1,0,0,0,0,0,2015-02-24,R1762_2015-02-24
2062,2015-02-24 15:31:17,R1762,0,1,0,0,0,0,0,2015-02-24,R1762_2015-02-24
3033,2015-02-26 13:12:11,R1762,0,1,0,0,0,0,0,2015-02-26,R1762_2015-02-26


In [0]:
unique_non_failed = non_failed[~non_failed['ID_Date'].isin(unique_failures)].ID_Date.unique().tolist()

In [0]:
B_train, B_test = train_test_split(unique_non_failed, test_size=0.2)

In [37]:
len(B_train)

109091

In [0]:
XB_train = X_train + B_train

In [39]:
len(XB_train)

110828

In [0]:
XB_test = B_test + X_test

In [0]:
modeldata['ID_Date'] = modeldata.EquipmentID.astype('str') + '_' + modeldata.Date.astype('str')

In [0]:
train = modeldata[modeldata['ID_Date'].isin(XB_train)]

In [0]:
test = modeldata[modeldata['ID_Date'].isin(XB_test)]

In [0]:
X_train=train[['Engine_Feature', 'Descriptor_Fuel', 'Engine_Exhaust', 'Engine_Coolant', 'Wheel_Sensor', 'NOx_limits']]
y_train=train['Target']

In [0]:
X_test=test[['Engine_Feature', 'Descriptor_Fuel', 'Engine_Exhaust', 'Engine_Coolant', 'Wheel_Sensor', 'NOx_limits']]
y_test=test['Target']

In [0]:
clf=RandomForestClassifier(n_estimators=50)

In [0]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)